In [5]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['lines.color']='k'
mpl.rcParams['lines.linestyle']='solid'
#mpl.rcParams['hist.bins'] = 50
%matplotlib inline
mpl.style.use('seaborn')

import pandas as pd
import pickle
import seaborn as sns

from src.ebm.plot import plot_shape_function

In [18]:
ebm_dir = 'models/ebm/'

with open(f'{ebm_dir}feature_names_all.pkl', 'rb') as f:
    feature_names = pickle.load(f)

num_cols = 0

# load raw data from ebm, gaminet, snam, i.e. feature importances, shape functions
with open(f'{ebm_dir}120f_shape_data.pkl', 'rb') as f:
    shape_data_ebm = pickle.load(f)

In [16]:
shape_data_ebm[0]['names']

[-7.413395741307348,
 -2.433973992481324,
 -2.09431312267043,
 -1.899102399620539,
 -1.76312458126635,
 -1.6594473525385214,
 -1.5724899187530972,
 -1.4998944497462876,
 -1.434819480053151,
 -1.3778950631001359,
 -1.3254014631525672,
 -1.280672708343887,
 -1.2393726791105937,
 -1.2003836282956861,
 -1.1628528777588993,
 -1.1282979876421355,
 -1.0962718470962516,
 -1.0662316690558513,
 -1.038197627018281,
 -1.0119291152834653,
 -0.9862505672767137,
 -0.961462440449393,
 -0.93857867967443,
 -0.9146330398565373,
 -0.8925641279961491,
 -0.8716993344482973,
 -0.8511269282311149,
 -0.8301083653983738,
 -0.811571230317139,
 -0.7923476251652422,
 -0.7737013811967219,
 -0.7554046303812462,
 -0.7375776069709241,
 -0.7206189072728162,
 -0.7025945411073831,
 -0.6863296416188635,
 -0.6693723080977225,
 -0.6540268237815923,
 -0.6390202151148832,
 -0.6226490200318239,
 -0.607104534190096,
 -0.5911274794496808,
 -0.577625344030466,
 -0.5626683179689443,
 -0.5479578622446117,
 -0.5326896998577378,
 -0.

In [ ]:
plot_shape_function(shape_data_ebm[0]['scores'], feature_name=feature_names, num_cols=num_cols, scaler_dict={})

In [ ]:
# plot top 20 shape functions ordered by importance
# TO-DO: plot ebm functions by importance (how?)